In [1]:
import numpy as np
from scipy.optimize import curve_fit    # フィッティング用
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import os
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import tensorflow as tf
import time
import csv

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' #意味はわからん

Using TensorFlow backend.


In [7]:
start = time.time()

# ガウシアンビームの関数の定義
def gaussian_beam(x,a,b,c,d):
    return  a * np.exp(-2*(x-b)*(x-b)/c/c) + d

def createdata(N, NOISE, x, y, tx, ty, fit_param_x, fit_param_y):
    start = time.time()
    
    # ガウシアンビームのパラメータ
    i0 = 1.0
    x0 = 0.0
    y0 = 0.0
    w0 = 5.0
    h0 = 0.0
    param_ini_x = np.array([i0, x0, w0, h0])  # フィッティングの初期値 (ここではデータから推定は行わない)
    param_ini_y = np.array([i0, y0, w0, h0])
    
    k = 1

    for n in range(N):        # 繰り返し精度を調べるために各ノイズ割合でN回実行
        
        center = data_size/2
        # x配列とy配列
        x_array = np.arange(-center, center, 1.0)                         # x配列
        y_array = np.arange(-center, center, 1.0)                         # y配列
        nx = len(x_array)
        ny = len(y_array)
        intensity = np.zeros((nx, ny))                            # ノイズを含まない2次元強度分布
        x0 = x0 + center
        y0 = y0 + center
        for i in range(nx):
            for j in range(ny):
                intensity[i][j] = i0 * np.exp(-2*((i-x0)*(i-x0) + (j-y0)*(j-y0))/w0/w0)
        x0 = x0 - center
        y0 = y0 - center
        # 最大強度を取る位置における強度プロファイル
        profile_x = np.zeros(nx)
        profile_y = np.zeros(ny)
    

        # 2次元の強度分布にノイズを付与
        noise = (np.random.rand(nx*ny)-0.5)*i0*NOISE*0.01   #プラスマイナスNOISE%のノイズ(一様分布), (np.random.rand(nx*ny)-0.5)*2の部分が-1から1までの乱数になる
        noise = noise.reshape((nx,ny))
        intensity_noise = intensity + noise

        # 最大値の探索 & その位置の強度プロファイルの取得
        idx = np.unravel_index(np.argmax(intensity_noise), intensity_noise.shape)
        profile_x= intensity_noise[:,idx[1]]
        profile_y = intensity_noise[idx[0],:]
        x[n] = profile_x
        y[n] = profile_y
        tx[n] = (i0, x0 + 1.0, w0)

        # x方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, x_array, profile_x, p0=param_ini_x, maxfev=2000)
        fit_param_x[n][0] = param[0]
        fit_param_x[n][1] = param[1]
        fit_param_x[n][2] = param[2]
        fit_param_x[n][3] = param[3]
        # y方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, y_array, profile_y, p0=param_ini_y, maxfev=2000)
        fit_param_y[n][0] = param[0]
        fit_param_y[n][1] = param[1]
        fit_param_y[n][2] = param[2]
        fit_param_y[n][3] = param[3]
        #中心位置の変化
        i = N*k / 20
        if n > i:
            x0 = x0 
            y0 = y0 
            k = k+1
            print(x0_N[n])
        elif n == N-1:
            elapsed_time = time.time() - start
            print ("経過時間:{0}".format(elapsed_time) + "[sec]")
            fit_ave = np.average(abs(fit_param_x.T[1]-tx.T[1]))
            fit_std = np.std(abs(fit_param_x.T[1]-tx.T[1]))
            fitting = "平均絶対誤差: %f ± %f" % (fit_ave, fit_std)
            print(fitting)
        else:
            x0 = x0
    return x, y, tx, ty

In [3]:
train_num = 10000 #訓練データの数
test_num = 1000 #テストデータの数
data_size = 100 #配列の大きさ
NOISE = 30
x_test = np.zeros((test_num,data_size))
y_test = np.zeros((test_num,data_size))
t_test = np.zeros((test_num, 3))
ty = np.zeros((test_num, 3))
fit_param_x = np.zeros((test_num, 4))
fit_param_y = np.zeros((test_num, 4))

#createdata(test_num, NOISE, x_test, y_test, t_test, ty, fit_param_x, fit_param_y)

In [1]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)

In [4]:
from createdatas import createdata
createdata(test_num, NOISE, data_size, x_test, y_test, t_test, ty, fit_param_x, fit_param_y)
print(t_test)

経過時間:19.90097689628601[sec]
平均絶対誤差: 0.111801 ± 0.084451
[[1.  0.  5. ]
 [1.  0.  5. ]
 [1.  0.  5. ]
 ...
 [1.  1.9 5. ]
 [1.  1.9 5. ]
 [1.  1.9 5. ]]


In [8]:
#10,000データの確認
test_num = 100 #データ数
data_size = 100 #配列の大きさ
fit_param_x = np.zeros((test_num, 4))
fit_param_y = np.zeros((test_num, 4))
NOISE = 30

x = np.zeros((test_num, data_size))
y = np.zeros((test_num, data_size))
t = np.zeros((test_num, 3))
ty = np.zeros((test_num, 3))

createdata(test_num, NOISE, x, y, t, ty, fit_param_x, fit_param_y)

NameError: name 'x0_N' is not defined